In [3]:
class Signal:
    '''
        Objet Signal correspondant à une courbe émise lorsqu'un camion est passé sur les capteurs
    '''
    
    def __init__(self, name, data, lookahead=15, delta=0.2, temp=False):
        '''
            Initialiation à la création d'une instance 'Signal'.
            Si temp=True, self.peaks et self.deletion_limite n'existe pas.
            Sinon, cela permet le calcul automatique des pics du signal, ainsi que la suppresion de ceux
            inutils pour notre analyse.
            __________
            
            param:
                - (name : str) Nom du signal
                - (data : np.array) Tableau de donnée du signal
                - (temp : bool) Définit la temporalité du signal
            __________
            
            return: (Objet Signal)
        '''
        
        self.name = name
        self.data = data
        if not temp:
            self.peaks = self.calculate__peaks(lookahead, delta) # list des pics
            self.deletion_limit = self.deletion_band(deletion=False) # float de la limit positive
        
    
    def deletion_band(self, pourcentage=0.4, nbr_std=3, deletion=False):
        '''
            Calcul de la zone de pics du signal à supprimer.
            Si deletion = True, les pics inclus dans la zone seront supprimés. Pas dans le cas contraire.
            __________
            
            param:
                - (pourcentage : float) Pourcentage du signal à analyse pour calculer la zone
                - (nbr_std : int) Multiplicateur d'écart-type pour égalir la zone
                - (deletion : bool) True pour supprimer les points False pour simplement calculer la zone.
            __________
            
            return: (float) Limite positive de la zone
        '''
        
        deletion_limit = self.data.iloc[:int(pourcentage*len(self.data)),1].std()*nbr_std
        
        if deletion:
            peaks = list(self.peaks)
            for peak in peaks:
                if (peak.amplitude < deletion_limit) & (peak.amplitude > - deletion_limit): 
                    self.peaks.remove(peak)
    
        return deletion_limit
    
    
    def find_close_point(self, close_point, peak=None):
        '''
            Recherche du point le plus proche de 'close_point' sur le signal.
            __________
            param:
            
                - (close_point : DataFrame) Point dont on cherche la plus proche correspondance sur le signal
            __________
            
            return: (DataFrame) contenant le numéro d'échantillon, l'amplitude et la position du point
        '''
        
        columns=["sample_number", "time", "amplitude"]
        points = pd.DataFrame(columns=columns)
        
        for index, row in close_point.iterrows():
            if peak != None:
                if row.time < peak.time:
                    signal_cut = Signal('Tempo gauche', self.data[self.data.iloc[:,0] > peak.time], temp=True)
                else:
                    signal_cut = Signal('Tempo droite', self.data[self.data.iloc[:,0] < peak.time], temp=True)
            else:
                signal_cut = self
                
            time_diff = abs(row.time - signal_cut.data.iloc[:,0])
            amplitude_diff = abs(row.amplitude - signal_cut.data.iloc[:,1])

            sum_diff = time_diff + amplitude_diff
            
            if sum_diff.min() > 0.4:
                new_df = pd.DataFrame([[0, 0, row.amplitude]])
            
            else:
                new_df = signal_cut.data.loc[[sum_diff.idxmin()],:].reset_index()
            
            new_df.columns = columns
            
            points = points.append(new_df, ignore_index=True)
        return points
    
    
    def find_symetric_point(self, first_point, peak):
        '''
            Recherche du point symétrique à 'first_point' sur le signal par rapport au 'peak'.
            __________
            
            param:
                - (first_point : DataFrame) Premier point sur le signal dont on cherche le symétrique
                - (peak : Pic) Pic représentant le centre de symétrie sur le signal
            __________
            
            return: (DataFrame) contenant le numéro d'échantillon, l'amplitude et la position du point symétrique
        '''
        
        points = pd.DataFrame(columns=["sample_number", "time", "amplitude"])
        
        for fp in first_point.iterrows(): 
            if fp[1].time < peak.time:
                signal_cut = Signal('Tempo gauche', self.data[self.data.iloc[:,0] > peak.time], temp=True)
            else:
                signal_cut = Signal('Tempo droite', self.data[self.data.iloc[:,0] < peak.time], temp=True)

            points = points.append(signal_cut.find_close_point(pd.DataFrame(fp[1]).T), ignore_index=True)
        
        return points
    
    
    def peaks_list(self, kind):
        '''
            Récupère tous les pics du signal du genre 'kind' (max, min ou all).
            __________
            
            param:
                - (kind : str) String ('max', 'min' ou 'all') dépendant des pics que l'on souhaite récupérer.
            __________
            
            return: (DataFrame) contenant l'ensemble des pics du signal du genre 'kind' selectionné. Affiché via Pic.to_df()
        '''
        peaks_df = pd.DataFrame()
        for peak in self.peaks:
            if peak.kind == kind:
                peaks_df = peaks_df.append(peak.to_df())
            elif kind == "all":
                peaks_df = peaks_df.append(peak.to_df())
        
        return peaks_df.reset_index(drop=True)
        
    
    def calcultate__half_max(self):
        '''
            Calcul des mi-hauteurs pour l'ensemble des pics contenus dans le signal
            __________
            
            return: (DataFrame) contenant les positions des points de mi-hauteurs calculés (time et amplitude)
        '''
        peaks_df = pd.DataFrame()
        
        for peak in self.peaks:
            peaks_df = pd.concat([peaks_df, peak.calcultate__half_max])

        return peaks_df
    
    
    def calculate__peaks(self, lookahead=15, delta=0.2):
        '''
            Detection de tous les pics du signal.
            __________
            
            param:
                - (lookahead : int) Distance entre le pic actuel déterminé et le candidat suivant
                - (delta : float) Différence minimum entre un pics et le candidat suivant
            __________
            
            return: (list) contenant un objet Pic par pic
        '''
        peaks_list = []
        
        peaks = peakdetect(self.data.iloc[:, 1],self.data.iloc[:, 0], lookahead=lookahead, delta=delta)
        for kind in peaks:
            if kind == peaks[0]:
                kind_name = 'max'
            else:
                kind_name = 'min'
                
            for sig in kind:
                peaks_list.append(Pic(self, sig[0], sig[1], kind_name))
        
        return peaks_list
    
    
    def show(self):
        '''
            Affichage de la courbe du signal, les limits + et - de la zone à supprimer,
            les pics et les points à mi-hauteur et leur symétrique.
            __________
            
            return:  plot
        '''
        plt.figure(figsize=(15, 10))

        ## Courbe de capteurs
        plt.plot(self.data.loc[:,'time'], self.data.loc[:, 'amplitude'])
        
        ## Zone de suppression des pics
        plt.plot([self.data.iloc[0,0], self.data.iloc[-1,0]], [self.deletion_limit, self.deletion_limit])
        plt.plot([self.data.iloc[0,0], self.data.iloc[-1,0]], [-self.deletion_limit, -self.deletion_limit])

        ## Detections des maximas et minimas
        # Pics
        plt.scatter(self.peaks_list('all')['time'], self.peaks_list('all')['amplitude'])
        # Largeur
        plt.scatter(self.peaks_list('all')['time p1'], self.peaks_list('all')['amplitude p1'])
        plt.scatter(self.peaks_list('all')['time p2'], self.peaks_list('all')['amplitude p2'])

        plt.legend(['Signal', 'Limit +', 'Limit -', 'Pics', '1er point largeur', '2nd point largeur'])
        plt.xlabel("Temps")
        plt.ylabel("Amplitude")

        plt.show()
        
        
    def to_df(self):
        '''
        
        '''
        all_peaks = pd.DataFrame()
        for index, peak in enumerate(self.peaks):
            keep_col = ["time", "amplitude", "fwhm"]
            df = peak.to_df()[keep_col]
            
            new_columns = []
            for col in df.columns:
                new_columns.append(str(col) + '_' + str(index))
                
            df.columns = new_columns
            all_peaks = pd.concat([all_peaks, df], axis=1)
            
        return all_peaks
        

In [3]:
class Pic:
    '''
        Objet Pic correspondant à un maximum ou un minimum d'un objet Signal
    '''
    
    def __init__(self, signal, time, amplitude, kind):
        '''
            Initialiation à la création d'une instance 'Pic'.
            Permet le calcul automatique du FWHM (Full Weight at Half Maximum) et la détermination
            du numéro d'échantillon sur la courbe.
            __________
            
            param:
                - (signal : Signal) Instance 'Signal' à laquelle le pic est rataché
                - (time : float) Position dans le temps à laquelle le pic est placé sur le signal
                - (amplitude : float) Position en amplitude à laquelle le pic est placé sur le signal
                - (kind : str) Genre du pic ('max' ou 'min')
            __________
            
            return: (Objet Pic)
        '''
        
        self.signal = signal
        self.time = time
        self.amplitude = amplitude
        self.kind = kind
        self.sample_number = self.find_sample_number()
        self.fwhm = self.calculate__fwhm() # float de la lagueur à mi-hauteur du pic
        
    
    def to_df(self):
        '''
            Conversion des informations de l'instance 'Pic' en DataFrame
            ---
            Informations données :
                - time
                - amplitude
                - sample_number
                - time p1
                - amplitude p1
                - time p2
                - amplitude p2
                - fwhm
                - kind
                - signal_name
            __________
            
            return: (DataFrame) contenant l'ensemble des informations du pic.
        '''
        
        if self.fwhm != None:
            df = pd.DataFrame(data=[[self.time, self.amplitude, self.sample_number, self.fwhm[0].loc['time',:][0],\
                                 self.fwhm[0].loc['amplitude',:][0], self.fwhm[1].loc['time',:][0],\
                                 self.fwhm[1].loc['amplitude',:][0], self.fwhm[2].loc[:,'fwhm'][0] , self.kind,\
                                 self.signal.name]],\
                        columns=["time", "amplitude", "sample_number", "time p1", "amplitude p1", "time p2",\
                                 "amplitude p2", "fwhm", "kind", "signal_name"])
        else:
            df = pd.DataFrame(data=[[self.time, self.amplitude, self.sample_number, np.NaN, self.kind,\
                                 self.signal.name]],\
                        columns=["time", "amplitude", "sample_number", "fwhm", "kind", "signal_name"])
        return df
    
    
    def calcultate__half_max(self):
        '''
            Calcul des mi-hauteurs pour l'ensemble des pics contenus dans le signal
            __________
            
            return: (DataFrame) contenant les positions des points de mi-hauteurs calculés (time et amplitude)
        '''
        
        middle_time = self.time
        middle_amplitude = self.amplitude - self.amplitude/2

        return pd.DataFrame(data = [[middle_time, middle_amplitude]], columns=["time", "amplitude"])
    
    
    def find_sample_number(self):
        '''
            Retrouver le numéro d'échantillon du Pic se rapportant au Signal
            __________
            
            return: (int) correspondant au numéro d'échantillon
        '''
        
        return self.signal.data[(self.signal.data.loc[:,'time'] == self.time) &\
                                (self.signal.data.loc[:,'amplitude'] == self.amplitude)].index[0]
    
    
    def calculate__fwhm(self):
        '''
            Calcule du FWHM (Full Weight at Half Maximum)
            __________
            
            return: (tuple - 3 DataFrame) contenant le 1er point de la mi-hauteur, le 2nd point de la mi-hauteur et la largeur entre ces 2 points.
        '''
        
        result = pd.DataFrame()
        
        line = self.amplitude
        max_all = self.signal.data.amplitude.max()
        min_all = self.signal.data.amplitude.min()
        
        plt.plot(self.signal.data)
        
        if self.kind == "max":
            while(line > min_all):
                line -= 0.15
                df = pd.DataFrame([[self.time, line]], columns=["time", "amplitude"])
                point_x = self.signal.find_close_point(df, self)
                point_y = self.signal.find_symetric_point(point_x, self)
                result = result.append([[point_y.time.values[0] - point_x.time.values[0]]])
                
                plt.scatter(point_x.amplitude.values[0], point_x.time.values[0])
                plt.scatter(point_y.amplitude.values[0], point_y.time.values[0])
        else:
            while(line < max_all):
                line += 0.15
                df = pd.DataFrame([[self.time, line]], columns=["time", "amplitude"])
                point_x = self.signal.find_close_point(df)
                point_y = self.signal.find_symetric_point(point_x, self)
                result = result.append([[point_y.time.values[0] - point_x.time.values[0]]])
                
        plt.show()
#         actuel_point_x = self.signal.data.loc[[self.sample_number],:]
#         actuel_point_y = self.signal.data.loc[[self.sample_number],:]

#         next_point_x = self.signal.data.loc[[self.sample_number-1],:]
#         next_point_y = self.signal.find_symetric_point(next_point_x, self)
        
#         if self.kind == "max":
#             while (actuel_point_x.amplitude.values[0] > next_point_x.amplitude.values[0]) & (actuel_point_y.amplitude.values[0] >= next_point_y.amplitude.values[0]):
#                 df = df.append([[actuel_point_y.time.values[0] - actuel_point_x.time.values[0]]])
                
#                 actuel_point_x = next_point_x
#                 actuel_point_y = next_point_y
                
#                 next_point_x = self.signal.data.loc[[actuel_point_x.index[0]-1],:]
#                 next_point_y = self.signal.find_symetric_point(next_point_x, self)
#         else:
#             while (actuel_point_x.amplitude.values[0] < next_point_x.amplitude.values[0]) & (actuel_point_y.amplitude.values[0] <= next_point_y.amplitude.values[0]):
#                 df = df.append([[actuel_point_y.time.values[0] - actuel_point_x.time.values[0]]])

#                 actuel_point_x = next_point_x
#                 actuel_point_y = next_point_y
                
#                 next_point_x = self.signal.data.loc[[actuel_point_x.index[0]-1],:]
#                 next_point_y = self.signal.find_symetric_point(next_point_x, self)
        

        return result.reset_index(drop=True)
                    
#         if self.kind == "max":
#             # Calcul mi-hauteur
#             half_max = self.calcultate__half_max()
#             point = self.signal.find_close_point(half_max)
#             first_point = pd.DataFrame(point).T
#             second_point = pd.DataFrame(self.signal.find_symetric_point(point, self)).T

#             # Calcul de la largeur
#             largeur = pd.DataFrame(np.transpose(np.abs(np.array(second_point.loc['time',:]) - np.array(first_point.loc['time',:]))), columns=["fwhm"])

#             result = (first_point, second_point, largeur)
#         else:
#             result = None
        
#         return result